In [9]:
#import dependcies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc
from sqlalchemy.sql import label

from flask import Flask, jsonify

In [10]:
#set up db
engine = create_engine("sqlite:///hawaii.sqlite")

In [11]:
#reflect db
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [12]:
#reference tables
Measurement = Base.classes.measurements
Stations = Base.classes.stations

AttributeError: stations

In [13]:
#create session
session = Session(bind = engine)

In [14]:
#query last year of prcp data
results = session.query(Measurement.date, Measurement.precip).filter(Measurement.date >= '2017-01-01')

AttributeError: type object 'measurements' has no attribute 'precip'

In [15]:
#create date & prcp df
data = {'date': [], 'precip': []}

for row in results:
    data['date'].append(row.date)
    data['precip'].append(row.precip)

# create pandas df from dict
prcp_df = pd.DataFrame(data['precip'], columns = ['prcp'], index = data['date'])

NameError: name 'results' is not defined

In [16]:
prcp_df.head()

NameError: name 'prcp_df' is not defined

In [17]:
plt.style.use('ggplot')
fig = plt.figure()

prcp_df.plot(figsize = (18,8), color='blue', rot = 340 )
plt.show()

NameError: name 'prcp_df' is not defined

In [19]:
#Station Analysis
total_stations = session.query(func.count(stations.station)).all()

NameError: name 'stations' is not defined